In [1]:
!pip install transformers sentencepiece sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.3 MB/s eta 0:00:00


In [2]:
! pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3


In [3]:
import os
from tqdm.auto import tqdm

import pandas as pd
import torch
from tqdm import tqdm
from typing import List, Union, Tuple, Dict
from datasets import load_dataset
import json
import logging

from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    T5Tokenizer,
    
    M2M100ForConditionalGeneration,
    NllbTokenizerFast,
    BartTokenizerFast,
    T5TokenizerFast,
    BartForConditionalGeneration,
    PreTrainedTokenizerFast,
    PreTrainedModel,
)

2024-05-12 06:32:19.561331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 06:32:19.561431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 06:32:19.698459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Training datasets preparation:

In [6]:
en_paradetox_content = load_dataset("s-nlp/en_paradetox_content")
paradetox = load_dataset("s-nlp/paradetox")
paranmt_for_detox = load_dataset("s-nlp/paranmt_for_detox")
multilingual_paradetox = load_dataset("textdetox/multilingual_paradetox")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating en split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating ru split:   0%|          | 0/400 [00:00<?, ? examples/s]

...

Generating es split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating am split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating zh split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating ar split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating hi split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [7]:
# Concatinating train and validation parts of the first dataset:

en_paradetox_content = pd.DataFrame(en_paradetox_content["train"])
paradetox_df = pd.DataFrame(paradetox["train"])
paranmt_for_detox_df = pd.DataFrame(paranmt_for_detox["train"])
eng_multilingual_paradetox_df = pd.DataFrame(multilingual_paradetox["en"])

In [8]:
en_paradetox_content = en_paradetox_content.drop(en_paradetox_content[en_paradetox_content["match"] == False].index)
en_paradetox_content = en_paradetox_content.drop("match", axis=1)

columns_titles = ["toxic","neutral"]
en_paradetox_content = en_paradetox_content.reindex(columns=columns_titles)

en_paradetox_content = en_paradetox_content.reset_index(drop=True)
en_paradetox_content = en_paradetox_content.rename(columns={"toxic": "toxic_sentence", "neutral": "neutral_sentence"})

en_paradetox_content.head(3)

,toxic_sentence,neutral_sentence
0,so sad the voters in her district are stupid e...,so sad the voters in her district are stupid e...
1,trump is a liar and a psychotic .,Trump is indifferent and not into his senses.
2,professors who sit in stinky old buildings dre...,professors who sit in an old buildings dreamin...


In [9]:
paradetox_df = paradetox_df.rename(columns={"en_toxic_comment": "toxic_sentence", "en_neutral_comment": "neutral_sentence"})
paradetox_df.head(3)

,toxic_sentence,neutral_sentence
0,he had steel balls too !,he was brave too!
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to..."


In [10]:
paranmt_for_detox_df = paranmt_for_detox_df.rename(columns={"toxic_comment": "toxic_sentence", "neutral_comment": "neutral_sentence"})
paranmt_for_detox_df.head(3)

,toxic_sentence,neutral_sentence
0,Shut it off!,turn it off!
1,Damned strange ones.,pretty weird.
2,Come on. I know I was a bit of a jerk back in ...,"I know I was quite a bastard then, but I've gr..."


In [11]:
eng_multilingual_paradetox_df = eng_multilingual_paradetox_df.rename(columns={"toxic_sentence": "toxic_sentence", "neutral_sentence": "neutral_sentence"})
eng_multilingual_paradetox_df.head(3)

,toxic_sentence,neutral_sentence
0,"then all of a sudden i see her , shes now got ...","All of a sudden i see her, she is all grown up."
1,My page should be protected first so that wort...,My page should be protected first so that unpl...
2,You made a mistake you ass.,You made a mistake.


In [12]:
eng_df = pd.concat([en_paradetox_content, paradetox_df, paranmt_for_detox_df, eng_multilingual_paradetox_df], axis=0)
eng_df

,toxic_sentence,neutral_sentence
0,so sad the voters in her district are stupid e...,so sad the voters in her district are stupid e...
1,trump is a liar and a psychotic .,Trump is indifferent and not into his senses.
2,professors who sit in stinky old buildings dre...,professors who sit in an old buildings dreamin...
3,i hope i live long enough to see at least thes...,i hope i live long enough to see at least thes...
4,"unkind , uncharitable , mean , small minded , ...","Unkind, uncharitable, mean, unlearned"
...,...,...
395,at least one party gives a crap about humans a...,at leas one party pays attention about humans ...
396,parliament is a lame duck .,Parliament is dull
397,"side - note , why the fuck does everything the...","side-note , why does everything the us make ha..."
398,"You're not being helpful, either, tiny-dick.","You're not being helpful, either."


In [13]:
eng_df["lang"] = "en"
eng_df = eng_df.reset_index(drop=True)
eng_df

,toxic_sentence,neutral_sentence,lang
0,so sad the voters in her district are stupid e...,so sad the voters in her district are stupid e...,en
1,trump is a liar and a psychotic .,Trump is indifferent and not into his senses.,en
2,professors who sit in stinky old buildings dre...,professors who sit in an old buildings dreamin...,en
3,i hope i live long enough to see at least thes...,i hope i live long enough to see at least thes...,en
4,"unkind , uncharitable , mean , small minded , ...","Unkind, uncharitable, mean, unlearned",en
...,...,...,...
49287,at least one party gives a crap about humans a...,at leas one party pays attention about humans ...,en
49288,parliament is a lame duck .,Parliament is dull,en
49289,"side - note , why the fuck does everything the...","side-note , why does everything the us make ha...",en
49290,"You're not being helpful, either, tiny-dick.","You're not being helpful, either.",en


# Trainig the model on the English corpus

In [14]:
lang_id_mapping = {
        "ru": "rus_Cyrl",
        "en": "eng_Latn",
        "am": "amh_Ethi",
        "es": "spa_Latn",
        "uk": "ukr_Cyrl",
        "zh": "zho_Hans",
        "ar": "arb_Arab",
        "hi": "hin_Deva",
        "de": "deu_Latn",
}

In [15]:
from IPython.display import display

class DetoxDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, tokenizer, lang):
        self.data = data
        self.tokenizer = tokenizer
        self.tokenizer.src_lang = lang_id_mapping[lang]
        self.tokenizer.tgt_lang = lang_id_mapping["en"]

    def __getitem__(self, idx):

        source = self.tokenizer(
            self.data.iloc[idx].toxic_sentence,
            max_length=150,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer(
            self.data.iloc[idx].neutral_sentence,
            max_length=150,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        source["labels"] = target["input_ids"]

        return {k: v.squeeze(0) for k, v in source.items()}

    def __len__(self):
        return self.data.shape[0]

In [16]:
def paraphrase(
    text,
    model,
    tokenizer,
    n=None,
    max_length="auto",
    beams=5,
):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        # forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang],
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [17]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(eng_df, random_state=42, test_size=0.05)

train = pd.DataFrame(train)
val = pd.DataFrame(val)

# lang = train[train["lang"] == "zh"].lang[0]
lang = "en"
trainset = DetoxDataset(train, tokenizer, lang)
valset = DetoxDataset(val, tokenizer, lang)

In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir="model",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=5,  # 8 is too much
    weight_decay=1e-5,
    num_train_epochs=3, # use 3 or 5 epochs here
    learning_rate=1e-5,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=1,
    logging_steps=500,
    gradient_accumulation_steps=1,
    load_best_model_at_end=True,
)

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=valset,
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
# cf252e122c0341df14b13db72773987d36dd67ec
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,8.082100,0.150243
1000,0.135300,0.094243
1500,0.105600,0.089298
2000,0.099400,0.086875
2500,0.095700,0.085786
3000,0.095100,0.084549
3500,0.092300,0.083821
4000,0.089900,0.083257
4500,0.089900,0.082822
5000,0.089400,0.082283


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=8781, training_loss=0.5487573054775998, metrics={'train_runtime': 7425.5226, 'train_samples_per_second': 18.919, 'train_steps_per_second': 1.183, 'total_flos': 2.81822453991936e+16, 'train_loss': 0.5487573054775998, 'epoch': 3.0})

In [22]:
model.save_pretrained("/kaggle/working/model/")

In [54]:
!git clone https://huggingface.co/MOOsipenko/fine-tuned_T5_NLP_HW2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'fine-tuned_T5_NLP_HW2'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (20/20), 7.40 KiB | 1.48 MiB/s, done.
Filtering content: 100% (7/7), 3.69 GiB | 47.03 MiB/s, done.


In [24]:
model = T5ForConditionalGeneration.from_pretrained("/kaggle/working/fine-tuned_T5_NLP_HW2/model")
tokenizer = T5Tokenizer.from_pretrained("/kaggle/working/fine-tuned_T5_NLP_HW2/model/checkpoint-8500")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Predicting the English part of dev and test data

In [25]:
dev_indices_dict = {"zh": [0, 400], "es": [400, 800], "ru": [800, 1200], "ar": [1200, 1600],
               "hi": [1600, 2000], "uk": [2000, 2400], "de": [2400, 2800], "am": [2800, 3200],
               "en": [3200, 3600]}

test_indices_dict = {"uk": [0, 600], "hi": [600, 1200], "zh": [1200, 1800], "ar": [1800, 2400],
               "de": [2400, 3000], "en": [3000, 3600], "ru": [3600, 4200], "am": [4200, 4800],
               "es": [4800, 5400]}

In [26]:
dev_df = pd.read_csv("/kaggle/input/public-hw2/inputs_multilingual.tsv", sep="\t")
test_df = pd.read_csv("/kaggle/input/sample-submission/sample_submission_test.tsv", sep="\t")

In [27]:
dev_df_copy = dev_df.copy()
test_df_copy = test_df.copy()

In [28]:
# dev:

dev_inputs = dev_df.iloc[3200:3600, 0].to_list()
dev_preds = []

for text in tqdm(dev_inputs):
    dev_preds.append(paraphrase(text, model, tokenizer))

100%|██████████| 400/400 [09:57<00:00,  1.49s/it]


In [34]:
dev_df_copy.iloc[3200:3600, 1] = dev_preds

In [30]:
# test:

test_inputs = test_df.iloc[3000:3600, 0].to_list()
test_preds = []

for text in tqdm(test_inputs):
    test_preds.append(paraphrase(text, model, tokenizer))

100%|██████████| 600/600 [15:01<00:00,  1.50s/it]


In [31]:
test_df_copy.iloc[3000:3600, 1] = test_preds

In [33]:
test_df_copy.iloc[3000:3600, :]

,toxic_sentence,neutral_sentence,lang
3000,"mandated and "" right fucking now "" would be g...","mandated and "" right now "" would be good.",en
3001,&gt i wouldn 't care how ignorant you are in y...,&gt i wouldn 't care how ignorant you are in y...,en
3002,6. are you fucking insane ?,6. are you insane? I don't know.,en
3003,after all its hard to get a job if your stupid .,After all its hard to get a job if you are not...,en
3004,after all most of these dorks are millionares ...,after all most of these dorks are millionares ...,en
...,...,...,...
3595,what do you mean why don't you keep your nose ...,What do you mean why don't you keep your nose ...,en
3596,Lets not edit user pages and look like a fucki...,Let's not edit user pages and look like a moro...,en
3597,Stalking you little shit you need to stop foll...,Stalking you little thing you need to stop fol...,en
3598,For fuck's sake When are they gonna send that ...,"For fuck's sake, when are they gonna send that...",en


# Predicting the Russian part of dev and test data

In [35]:
def get_model(type: str) -> Tuple[PreTrainedModel, PreTrainedTokenizerFast]:
    """
    Returns a pre-trained model and tokenizer based on the specified type.

    Args:
        type (str): The type of model to retrieve.
        Valid options are "translator", "en_detoxifier", and "ru_detoxifier".

    Returns:
        (PreTrainedModel, PreTrainedTokenizer)
    Raises:
        ValueError: If an invalid type choice is provided.

    Examples:
        model, tokenizer = get_model("translator")
        model, tokenizer = get_model("en_detoxifier")
        model, tokenizer = get_model("ru_detoxifier")
    """
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

    model_types: Dict[str, Tuple[str, PreTrainedModel, PreTrainedTokenizerFast]] = {
        "translator": (
            "facebook/nllb-200-distilled-600M",
            M2M100ForConditionalGeneration,
            NllbTokenizerFast,
        ),
        "en_detoxifier": (
            "s-nlp/bart-base-detox",
            BartForConditionalGeneration,
            BartTokenizerFast,
        ),
        "ru_detoxifier": (
            "s-nlp/ruT5-base-detox",
            T5ForConditionalGeneration,
            T5TokenizerFast,
        ),
    }

    if type not in model_types:
        raise ValueError("Invalid type choice")

    model_name, ModelClass, TokenizerClass = model_types[type]

    logging.info(f"Loading {type} model: {model_name}")

    model = ModelClass.from_pretrained(model_name).eval().to(device)
    tokenizer = TokenizerClass.from_pretrained(model_name)

    return model, tokenizer

In [36]:
def detoxify_batch(
    texts: List[str],
    model: Union[BartForConditionalGeneration, T5ForConditionalGeneration],
    tokenizer: PreTrainedTokenizerFast,
    batch_size: int = 32,
) -> List[str]:
    """
    Detoxify a batch of texts.

    Args:
        texts (List[str]): The list of texts to detoxify.
        model (Union[BartForConditionalGeneration, T5ForConditionalGeneration]): The detoxification model.
        tokenizer (PreTrainedTokenizerFast): The tokenizer for the detoxification model.
        batch_size (int, optional): The batch size for detoxification. Defaults to 32.

    Returns:
        List[str]: The detoxified texts.
    """
    detoxified = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Detoxifying"):
        batch = texts[i : i + batch_size]
        batch_detoxified = model.generate(
            **tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(
                model.device
            )
        )
        detoxified.extend(
            tokenizer.decode(tokens, skip_special_tokens=True)
            for tokens in batch_detoxified
        )
    return detoxified

In [37]:
model, tokenizer = get_model("ru_detoxifier")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

In [38]:
# dev:

dev_text = dev_df_copy.iloc[800:1200, 0].to_list()
detoxified_dev = detoxify_batch(dev_text, model, tokenizer, 16)

Detoxifying:   0%|          | 0/25 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Detoxifying: 100%|██████████| 25/25 [00:09<00:00,  2.54it/s]


In [39]:
dev_df_copy.iloc[800:1200, 1] = detoxified_dev

In [40]:
#test:

test_text = test_df_copy.iloc[3600:4200, 0].to_list()
detoxified_test = detoxify_batch(test_text, model, tokenizer, 16)

Detoxifying:   0%|          | 0/38 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Detoxifying: 100%|██████████| 38/38 [00:12<00:00,  2.99it/s]


In [41]:
test_df_copy.iloc[3600:4200, 1] = detoxified_test

# Predicting the rest languages

In [42]:
def detoxify(
    text: str,
    stopwords: List[str],
    remove_all_terms: bool = False,
    remove_no_terms: bool = False,
) -> str:

    if remove_no_terms:
        return text
    if remove_all_terms:
        return ""
    tokens = [
        token
        for token in text.replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("?", "").replace("!", "").split()
        if not stopwords or token.lower().strip() not in stopwords
    ]
    return " ".join(tokens)

In [43]:
dev_indices_dict = {"zh": [0, 400], "es": [400, 800], "ru": [800, 1200], "ar": [1200, 1600],
               "hi": [1600, 2000], "uk": [2000, 2400], "de": [2400, 2800], "am": [2800, 3200],
               "en": [3200, 3600]}

In [44]:
# dev:

dev_zh_deleting = dev_df_copy.iloc[0:400, 0].to_list()
dev_es_deleting = dev_df_copy.iloc[400:800, 0].to_list()
dev_ar_deleting = dev_df_copy.iloc[1200:1600, 0].to_list()
dev_hi_deleting = dev_df_copy.iloc[1600:2000, 0].to_list()
dev_uk_deleting = dev_df_copy.iloc[2000:2400, 0].to_list()
dev_de_deleting = dev_df_copy.iloc[2400:2800, 0].to_list()
dev_am_deleting = dev_df_copy.iloc[2800:3200, 0].to_list()

dev_list_of_lists = [dev_zh_deleting, dev_es_deleting, dev_ar_deleting, dev_hi_deleting, dev_uk_deleting, dev_de_deleting, dev_am_deleting]
dev_language_list = ["zh", "es", "ar", "hi", "uk", "de", "am"]

In [45]:
df_list = []

for k, i in tqdm(enumerate(dev_list_of_lists)):
    
    texts = []
    stopwords = load_dataset("textdetox/multilingual_toxic_lexicon")[dev_language_list[k]]["text"]
    stopwords = set(stopwords)
    
    for j in i:
        texts.append(detoxify(j, stopwords))
        
    df_list.append(pd.DataFrame({"neutral": texts}))

0it [00:00, ?it/s]

Generating am split:   0%|          | 0/245 [00:00<?, ? examples/s]

Generating es split:   0%|          | 0/1195 [00:00<?, ? examples/s]

Generating ru split:   0%|          | 0/140517 [00:00<?, ? examples/s]

Generating uk split:   0%|          | 0/7356 [00:00<?, ? examples/s]

...

Generating zh split:   0%|          | 0/3839 [00:00<?, ? examples/s]

Generating ar split:   0%|          | 0/430 [00:00<?, ? examples/s]

Generating hi split:   0%|          | 0/133 [00:00<?, ? examples/s]

Generating de split:   0%|          | 0/247 [00:00<?, ? examples/s]

7it [02:24, 20.67s/it]


In [46]:
dev_df_copy.iloc[0:400, 1] = df_list[0].iloc[:, 0].to_list()
dev_df_copy.iloc[400:800, 1] = df_list[1].iloc[:, 0].to_list()
dev_df_copy.iloc[1200:1600, 1] = df_list[2].iloc[:, 0].to_list()
dev_df_copy.iloc[1600:2000, 1] = df_list[3].iloc[:, 0].to_list()
dev_df_copy.iloc[2000:2400, 1] = df_list[4].iloc[:, 0].to_list()
dev_df_copy.iloc[2400:2800, 1] = df_list[5].iloc[:, 0].to_list()
dev_df_copy.iloc[2800:3200, 1] = df_list[6].iloc[:, 0].to_list()

In [47]:
dev_df_copy = dev_df_copy.reset_index(drop=True)

In [48]:
dev_df_copy.to_csv("final_dev.tsv", sep="\t", index=False)

In [49]:
# test:

test_uk_deleting = test_df_copy.iloc[0:600, 0].to_list()
test_hi_deleting = test_df_copy.iloc[600:1200, 0].to_list()
test_zh_deleting = test_df_copy.iloc[1200:1800, 0].to_list()
test_ar_deleting = test_df_copy.iloc[1800:2400, 0].to_list()
test_de_deleting = test_df_copy.iloc[2400:3000, 0].to_list()
test_am_deleting = test_df_copy.iloc[4200:4800, 0].to_list()
test_es_deleting = test_df_copy.iloc[4800:5400, 0].to_list()

test_list_of_lists = [test_uk_deleting, test_hi_deleting, test_zh_deleting, test_ar_deleting, test_de_deleting, test_am_deleting, test_es_deleting]
dev_language_list = ["uk", "hi", "zh", "ar", "de", "am", "es"]

In [50]:
df_list = []

for k, i in tqdm(enumerate(test_list_of_lists)):
    
    texts = []
    stopwords = load_dataset("textdetox/multilingual_toxic_lexicon")[dev_language_list[k]]["text"]
    stopwords = set(stopwords)
    
    for j in i:
        texts.append(detoxify(j, stopwords))
        
    df_list.append(pd.DataFrame({"neutral": texts}))

7it [02:09, 18.50s/it]


In [51]:
test_df_copy.iloc[0:600, 1] = df_list[0].iloc[:, 0].to_list()
test_df_copy.iloc[600:1200, 1] = df_list[1].iloc[:, 0].to_list()
test_df_copy.iloc[1200:1800, 1] = df_list[2].iloc[:, 0].to_list()
test_df_copy.iloc[1800:2400, 1] = df_list[3].iloc[:, 0].to_list()
test_df_copy.iloc[2400:3000, 1] = df_list[4].iloc[:, 0].to_list()
test_df_copy.iloc[4200:4800, 1] = df_list[5].iloc[:, 0].to_list()
test_df_copy.iloc[4800:5400, 1] = df_list[6].iloc[:, 0].to_list()

In [52]:
test_df_copy = test_df_copy.reset_index(drop=True)

In [53]:
test_df_copy.to_csv("final_test.tsv", sep="\t", index=False)